# 1. Scraping text from a Kickstarter page

Load required libraries.

In [18]:
import nltk
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

Select a test hyperlink.

In [19]:
hyperlink = 'https://www.kickstarter.com/projects/636921542/mobile-phone-cup-holder-dock-and-stand-apple-and-a?ref=category'

Scrape the HTML content from the website pointed by the hyperlink.

In [20]:
doc = requests.get(hyperlink)
soup = BeautifulSoup(doc.text, 'html.parser')

Replace all line break markers with a whitespace.

In [21]:
parsed_text = soup.get_text().replace('\n', ' ')

Define functions to extract information on the two main sections: *About This Project* and *Risks and Challenges*.

In [22]:
def extract_about_project(text):
    # Extracts the 'About this project' blurb
    return ''.join(re.findall(
        r'(?<=About this project).*(?=Risks and challenges)',
        text
    ))

In [23]:
def extract_risks(text):
    # Extracts the 'Risks and challenges' blurb
    return ''.join(re.findall(
        r'(?<=Risks and challenges).*(?=Learn about accountability on Kickstarter)',
        text
    ))

Define a function to clean up the extracted sections.

In [24]:
def clean_up(messy_text):    
    # Shrinks all whitespace to a single space
    clean_text = re.sub(r'\s+', ' ', messy_text)
    
    # Removes leading and trailing whitespace
    clean_text = re.sub(r'^\s+|\s+?$', '', clean_text)
    
    # Removes the video warning in the scraped content
    warning_str = "You'll need an HTML5 capable browser to see this content. Play Replay with sound Play with sound 00:00 00:00"
    return clean_text.replace(warning_str, '')

Test the above functions on `parsed_text`.

In [25]:
clean_up(extract_about_project(parsed_text))

'The first and only vehicle cup holder mobile phone dock and stand. This dock and stand is very easy to use with just one hand, no more getting in the vehicle and have to find your charging or syncing cord then plug in and put your phone on your seat or your lap only to have to turn sharply or break and your phone slides in the crevice of the seat or the floor, no more getting a call and cannot find your phone, with the mount you simply get in and drop your phone on the mount and it stays put and where you can always see the display, once docked the phone charges and connects to your stereo.'

In [26]:
clean_up(extract_risks(parsed_text))

'Only risks could be timing issues from obtaining parts.'

Test the extraction functions on a more lengthy Kickstarter page.

In [27]:
hyperlink = 'https://www.kickstarter.com/projects/sbf/sculpto-the-worlds-most-user-friendly-desktop-3d-p?ref=recommended'

In [28]:
doc = requests.get(hyperlink)
soup = BeautifulSoup(doc.text, 'html.parser')
parsed_text = soup.get_text().replace('\n', ' ')

In [29]:
about_project_text = clean_up(extract_about_project(parsed_text))
about_project_text

"3D printers are one of the coolest and most efficient ways for you to make your ideas go from the drawing board and into real life.Two years ago we launched a new type of 3D printer on Kickstarter. We wanted to make the amazing world of 3D printing available to everyone - not just engineers and tech-savvy people. We believe everyone should be able to bring their ideas to life. We started delivering that dream a year ago and opened a world of 3D printing for regular people and schools all over Denmark. Since then we have refined our production and software while developing Sculpto+The result is a small 3D printer with one of the biggest print areas seen on a plug'n'play printer. We have maximized printing performance while eliminating the noise and inconveniences of 3D printing - we have made the perfect desktop 3D printer. And the best part: Everyone can use it! Sculpto+ is a small 3D printer. It only weighs 2.7 KG and is about the size of your household coffeemaker, but has a printin

In [30]:
risks_text = clean_up(extract_risks(parsed_text))
risks_text

"The development of the Sculpto+ is almost finalized. Due to the fact that we are already manufacturing our previous Sculpto 3D printer - with a lot similarities to the Sculpto+ and have printers operating in private homes, schools and institutions all over Denmark we are very confident we can bring the Sculpto+ to production. The past 4 months we have been testing 3 stages of different prototypes and are now testing 10 production prototypes that are manufactured in a similar way to the final product. Our results have been amazing and we truly believe this 3D printer and the upgraded app will be a game-changer within 3D printing for consumers and educational use. Take a look at our 'prototype gallery' to see more. We have already started lining the production up and believe it is realistic to deliver all or a big part (Depending on the Kickstarters outcome) of the Sculpto+ 3D printers before Christmas."

# 2. Feature engineering

## Define functions to extract features

Count the number of sentences in the *About This Project* section.

In [33]:
def tokenize_sentences(text):
    # Tokenizes text into sentences and returns them in a list
    return nltk.sent_tokenize(text)

In [34]:
len(tokenize_sentences(about_project_text))

84

Count the number of all-caps words in the *About This Project* section.

In [35]:
def count_all_caps(text):
    # Counts the number of words in all-caps
    return len(re.findall(r'[A-Z]{2,}', text))

In [36]:
count_all_caps(about_project_text)

8

Count the number of exclamation points in the *About This Project* section.

In [39]:
def count_exclamations(text):
    # Counts the number of exclamation marks present in the text
    return len(re.findall(r'!', text))

In [41]:
count_exclamations(about_project_text)

5

Count the number of words in the *About This Project* section.

In [42]:
def remove_punc(text):
    # Returns the text with punctuation removed
    return re.sub(r'[^\w\d\s]', '', text)

In [43]:
def tokenize_words(text):
    # Tokenizes text into words and returns them in a list
    return nltk.word_tokenize(remove_punc(text))

In [45]:
len(tokenize_words(about_project_text))

1664

Count the number of words per sentence in the *About This Project* section.

In [60]:
def compute_avg_words(text):
    return pd.Series(
        [len(tokenize_words(sentence)) for sentence in \
         tokenize_sentences(text)]
    ).mean()

In [61]:
compute_avg_words(about_project_text)

60.5

## Create a master function for feature engineering

Using a list of hyperlinks, process each page, extract text sections, compute features, and store the results in a Pandas DataFrame.

In [66]:
# Initialize empty DataFrames
about_project_df = pd.DataFrame(
    columns=['num_sents', 'num_all_caps', 'num_exclms', 'num_words', 
             'avg_words_per_sent']
)

risks_df = pd.DataFrame(
    columns=['num_sents', 'num_all_caps', 'num_exclms', 'num_words', 
             'avg_words_per_sent']
)

Define a function to extract both sections given a hyperlink.

In [57]:
def extract_sections(hyperlink):
    # Scrape the HTML content from the website pointed by the hyperlink
    doc = requests.get(hyperlink)
    soup = BeautifulSoup(doc.text, 'html.parser')
    
    # Replace all line break markers with a whitespace
    parsed_text = soup.get_text().replace('\n', ' ')
    
    # Extract and clean up both sections
    return (
        clean_up(extract_about_project(parsed_text)), 
        clean_up(extract_risks(parsed_text))
    )

In [63]:
def extract_features(text):
    # Compute all features and return them in a list
    sentence_count = len(tokenize_sentences(text))
    all_caps_count = count_all_caps(text)
    exclm_count = count_exclamations(text)
    word_count = len(tokenize_words(text))
    avg_words = compute_avg_words(text)
    return [sentence_count, all_caps_count, exclm_count, word_count, avg_words]

In [55]:
hyperlinks = [
    'https://www.kickstarter.com/projects/636921542/mobile-phone-cup-holder-dock-and-stand-apple-and-a?ref=category',
    'https://www.kickstarter.com/projects/sbf/sculpto-the-worlds-most-user-friendly-desktop-3d-p?ref=recommended'
]

In [67]:
# Process text for each hyperlink and store features for each section in
# separate DataFrames
for index, link in enumerate(hyperlinks):
    about_project_text, risks_text = extract_sections(link)
    about_project_df.loc[index] = extract_features(about_project_text)
    risks_df.loc[index] = extract_features(risks_text)

In [69]:
about_project_df

,num_sents,num_all_caps,num_exclms,num_words,avg_words_per_sent
0,2.0,0.0,0.0,121.0,60.500000
1,84.0,8.0,5.0,1664.0,19.809524


In [70]:
risks_df

,num_sents,num_all_caps,num_exclms,num_words,avg_words_per_sent
0,1.0,0.0,0.0,9.0,9.000000
1,6.0,0.0,0.0,154.0,25.666667
